# Type structure and interface of bond objects

One of the most defining features of lattice is connectivity between sites that is defined by **bond**. A bond therefore has its own type in `LatticePhysics.jl`, equipped with an abstract supertype as well as a shared interface.

The following tutorial aims on providing an overview over the type hierachy of bonds, introducing the abstract type `AbstractBond`, as well as discussing the interface functions for this abstract type by showing examples for the concrete struct `Bond`.

Note 1: It should be mentioned, that bonds are (at least in the scope of this library) synonimous with interactions, i.e. every interaction between objects on two sites is captured by one bond.

Note 2: It should further be mentioned, that a bond is similar to a directed link in a graph, for a unitcell or lattice, there is always the returning bond as well.


In [ ]:
using LatPhysBase

### Abstract type `AbstractBond{L,N}`

The abstract type that is supertype to all bond implementations is `AbstractBond`, parametric in two parameters `L` and `N`. These two parameters give the label type of the bond, `L`, as well as the number of Bravais lattice dimensions, `N`, of the surrounding Bravais lattice. The latter is used to determine if the bond points to a site within the current copy of the finite lattice / unitcell or a shifted copy along one of the Bravais lattice vectors.
The two parameters allow for multiple dispatch on different labels as well as different Bravais lattices, as demonstrated in the following example

In [ ]:
# this is a correct suptyping of bond types
AbstractBond{String,2} <: AbstractBond

In [ ]:
# correct, since Float64 is subtype of Real
AbstractBond{Float64,2} <: AbstractBond{L,2} where {L<:Real}

In [ ]:
# not correct, since String is not subtype of Real
AbstractBond{Float64,2} <: AbstractBond{L,2} where {L<:String}

In [ ]:
# not correct, since Bravais lattice dimensions don't agree
AbstractBond{Float64,2} <: AbstractBond{L,3} where {L<:Real}

### Concrete type `Bond{L,N} <: AbstractBond{L,N}`

For all explicit `Unitcell` and `Lattice` objects, a concrete bond type is needed. Although every user could in principle implement its own concrete bond type, a default implementation is provided within `LatticePhysics.jl`, the `mutable struct Bond` implementation.

`Bond` should be regarded as a subtype of `AbstractBond` and can therefore be instatiated so it should be used whenever a concrete bond implementation is needed, i.e. whenever objects are created.

In [ ]:
Bond{String,2} <: AbstractBond{String,2}

### Interface of `AbstractBond` - how to access information correctly

In order to gain a benefit from having an abstract supertype `AbstractBond`, one has to define a common interface that all concrete bond types that are subtype of `AbstractBond` have to implement. Then one can simply call this interface for a given (but unknown) bond object that is subtype of `AbstractBond` and rely on getting the correct results.

The interface of `AbstractBond` contains various functions, which are explained in the following.


##### 1. The constructor

The constructor interface is used to create new objects of a specified bond type. It is therefore demanding to pass the bond type explicitly, as well as further information regarding the bond. The constructor syntax is
```
function newBond(
            :: Type{B},
            from    :: Integer,
            to      :: Integer,
            label   :: L,
            wrap    :: NTuple{N,<:Integer}
        ) :: B where {L,N,B<:AbstractBond{L,N}}
```
This interface is overwritten by the concrete bond type `Bond` to
```
function newBond(
            ::Type{Bond{L,N}},
            from    :: Int64,
            to      :: Int64,
            label   :: L,
            wrap    :: NTuple{N,Int64}
        ) :: Bond{L,N} where {L,N}
```


Using this constructor in constructing a `Bond{String,2}` bond object could work as follows:

In [ ]:
b = newBond( Bond{String,2}, 32, 42, "mybond", (-1,0) )

In a practical application, the bond type could be determined by e.g. the `Unitcell` type (which is parametric in the bond type) and is only available by dispatch within the function. This way, the bond type is never known explicitly in the code but upon execution, julia can compile a type stable function for the specific type.




##### 2. Setter and getter for internal fields

For a given bond object, the most abundant operation will be to access the connecting sites or label information. This could be either a static lookup or an overwrite. The syntax of these operations only differs by a `!` to pronounce naming similarities.

All in all, there are four types of information accessible:
1. the index of site that the bond originates from can be accessed with interface functions `from` and `from!`
2. the index of site that the bond points to can be accessed with interface functions `to` and `to!`
3. the label can be accessed with interface functions `label` and `label!`
4. the wrap, i.e. which copy of unitcell/lattice is targeted, can be accessed with interface functions `wrap` and `wrap!`

All functions are demonstrated in the example below

In [ ]:
# print the bond origin index
println(  from(b)  )

# set the bond origin index to a new index
from!(b, 876)

# print the bond origin index again
println(  from(b)  )

In [ ]:
# print the bond destination index
println(  to(b)  )

# set the bond destination index to a new index
to!(b, 123)

# print the bond destination index again
println(  to(b)  )

In [ ]:
# print the bond label
println(  label(b)  )

# set the bond label to a new label
label!(b, "newlabel")

# print the bond label again
println(  label(b)  )

In [ ]:
# print the bond wrap
println(  wrap(b)  )

# set the bond wrap to a new wrap, i.e. point in another unitcell/lattice copy
wrap!(b, (-1,1))

# print the bond wrap again
println(  wrap(b)  )

##### 3. Checking periodicity / wrapping

A common distinction for bonds is whether they point into a different copy of the current unitcell / lattice or not. In terms of the `AbstractBond` interface, this corresponds to having non-zero elements in the `wrap`.

To simplify the checking of periodicity, a function called `isPeriodic` is implemented on the level of `AbstractBond` that checks the elements of `wrap` explicitly. The return value is a `Bool` that describes whether there are non-zero elements in `wrap` or not.

In [ ]:
b = newBond( Bond{String,2}, 32, 42, "mybond", (-1,0) )
println("b is periodic?: ",  isPeriodic(b)  )

In [ ]:
b = newBond( Bond{String,2}, 32, 42, "mybond", (0,0) )
println("b is periodic?: ",  isPeriodic(b)  )